<a href="https://colab.research.google.com/github/VickyCarlson/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:15 http://ppa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-09 13:26:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.96MB/s    in 0.2s    

2022-01-09 13:26:52 (4.96 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://bookrevs.s3.us-west-2.amazonaws.com/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35471030|R1C5OK9AND7PRP|B00AHK07V0|     175130663|Hunter's Moon (A ...|Digital_Ebook_Pur...|          5|            0|          0|   N|                Y|Shugak is like a ...|This is the most ...| 2015-08-31|
|         US|   26579324|R333RNBQMPVUFT|B014085WTQ|     859232728|Flying Toward For...|Digital_Ebook_Pur

In [5]:
# Create the vine_table DataFrame
from pyspark.sql.types import IntegerType
vine_df = df.select(["review_id", df["star_rating"].cast('double'), "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1C5OK9AND7PRP|        5.0|            0|          0|   N|                Y|
|R333RNBQMPVUFT|        5.0|            0|          0|   N|                Y|
|R2A2K0GTNSKXM1|        5.0|            0|          0|   N|                N|
|R365LAQ9REV876|        3.0|            0|          0|   N|                Y|
|R294OF3SNH6SWZ|        4.0|            0|          0|   N|                Y|
|R2OMRH58WYEEP9|        5.0|            0|          0|   N|                N|
| RBFXYUJZIH5JR|        5.0|            0|          0|   N|                Y|
|R2K2RSBNV42HU4|        5.0|            0|          0|   N|                N|
|R2CLRFFJ5HJSU3|        4.0|            0|          0|   N|                N|
| RS1O7V45AADDO|        4.0|            0|          0|   N|     

In [6]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
vine_total_df = vine_df.filter("total_votes>= 20")
vine_total_df.count()

70726

In [7]:
# Filter the new DataFrame create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
vine_helpful_votes_df = vine_total_df.filter("(helpful_votes/total_votes)>=0.5")
vine_helpful_votes_df.count()

54092

In [8]:
vine_yes_df = vine_helpful_votes_df.filter("vine=='Y'")
vine_yes_df.show()
vine_yes_df.count()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



0

In [9]:
vine_no_df = vine_helpful_votes_df.filter("vine=='N'")
vine_no_df.show()
vine_no_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HQLKREFKG25D|        5.0|           21|         21|   N|                Y|
| RWIL1XU6YM0U4|        5.0|           20|         20|   N|                N|
|R28SB7Q1WBJRFM|        5.0|           25|         27|   N|                N|
|R3PVKAWMZW55U8|        5.0|          106|        119|   N|                Y|
|R21JS1HIQV1H7W|        3.0|           17|         22|   N|                Y|
|R245IE9DATP3ZL|        3.0|           15|         23|   N|                Y|
| RB1Y3RO4J4854|        3.0|           23|         30|   N|                Y|
|R23P75H9VX5RLM|        5.0|           32|         36|   N|                Y|
|R3JCH5P63TIQ4E|        5.0|           19|         20|   N|                N|
| RYA7IZ8VHN5P8|        2.0|           37|         42|   N|     

54092

In [19]:
vine_no_df_ct = vine_no_df.count()
vine_no_df_ct

54092

In [21]:
vine_helpful_votes_df_ct= vine_helpful_votes_df.count()
vine_helpful_votes_df_ct

54092

In [22]:
total_reviews_ct = vine_helpful_votes_df_ct
total_reviews_ct

54092

In [39]:
Five_Star_No_df = vine_no_df.filter("star_rating==5.0")
Five_Star_No_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HQLKREFKG25D|        5.0|           21|         21|   N|                Y|
| RWIL1XU6YM0U4|        5.0|           20|         20|   N|                N|
|R28SB7Q1WBJRFM|        5.0|           25|         27|   N|                N|
|R3PVKAWMZW55U8|        5.0|          106|        119|   N|                Y|
|R23P75H9VX5RLM|        5.0|           32|         36|   N|                Y|
|R3JCH5P63TIQ4E|        5.0|           19|         20|   N|                N|
| RE754WU5TGN9E|        5.0|           39|         44|   N|                Y|
|  RVNRTIVFKBBI|        5.0|           23|         23|   N|                N|
|R31X4TF63XSPKJ|        5.0|           30|         35|   N|                Y|
|R109YPQMPNG6J3|        5.0|           20|         22|   N|     

In [42]:
Five_Star_No_df_ct = Five_Star_No_df.count()
Five_Star_No_df_ct

20918

In [45]:
Five_Star_Yes_df = vine_yes_df.filter("star_rating==5.0")
Five_Star_Yes_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [48]:
Five_Star_Yes_df_ct = Five_Star_Yes_df.count()
Five_Star_Yes_df_ct

0

In [50]:
Percent_Yes = (Five_Star_Yes_df_ct/total_reviews_ct)*100
Percent_Yes

0.0

In [51]:
Percent_No = (Five_Star_No_df_ct/total_reviews_ct)*100
Percent_No

38.67115285069881